# Super-quick LangGraph intro

This notebook will very briefly go through the basics of LangGraph.

<a target="_blank" href="https://colab.research.google.com/github/tolo/simple-rag-agent-demo/blob/main/super-quick-langgraph-intro.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a><br/>

![Let's build an agent](https://github.com/tolo/simple-rag-agent-demo/blob/main/images/llm-apps-2024.png?raw=true)

## Setup

### Install dependencies

In [ ]:
%pip install httpx~=0.28.1 openai~=1.57 --upgrade --quiet
%pip install python-dotenv~=1.0 docarray~=0.40.0 pypdf~=5.1 --upgrade --quiet
%pip install chromadb~=0.5.18 lark~=1.2 --upgrade --quiet
%pip install langchain~=0.3.10 langchain_openai~=0.2.11 langchain_community~=0.3.10 langchain-chroma~=0.1.4 --upgrade --quiet
%pip install langgraph~=0.2.56 --upgrade --quiet

# If running locally, you can do this instead:
#%pip install -r ../requirements.txt

### Load environment variables

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# If running in Google Colab, you can use this code instead:
# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

### Setup Chat Model

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
llm = ChatOpenAI(model_name="gpt-4o-mini",temperature=0.0)
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

## Basic LangChain / LangGraph concepts
Below are some core concepts that are common to both LangChain and LangGraph.

### Messages
Questions and instructions sent to an LLM are called messages. Messages come in different flavours, corresponding to different roles, for instance _**System, Human, AI, Tool**_ etc. Read more about messages [here](https://python.langchain.com/docs/concepts/messages/).

### Tools
Tools are functions that can be called by an LLM. Tools can be used to perform calculations, look up information, or perform other tasks. _**Tools**_ and _**tool calling**_ are the underpinnings of building autonomous _**agents**_. Read more about tools [here](https://python.langchain.com/docs/concepts/tools/).

## LangGraph concepts
Below are some concepts specific to LangGraph, related to modelling logic and behaviour as graphs of nodes and edges.

### State
To keep track of the state of the graph, we use a state object. A state object can be anything from a simple dictionary to a complex object. The state object is passed between nodes in the graph and is updated as the graph progresses. Read more about states [here](https://langchain-ai.github.io/langgraph/concepts/low_level/#state).

### Nodes
A node is a unit of work in a graph. A note can be implemented as a simple function or by using a class. Read more about nodes [here](https://langchain-ai.github.io/langgraph/concepts/low_level/#nodes).

### Edges
An edge is a connection - or a transition - between two nodes. An edge can be **_conditional_**, meaning that the transition is decided based on the state of the graph.

Read more about edges [here](https://langchain-ai.github.io/langgraph/concepts/low_level/#edges).

![Graph](https://github.com/tolo/simple-rag-agent-demo/blob/main/images/graph.png?raw=true)

## Let's build a simple hello-world-ish graph

In [ ]:
from langgraph.graph import MessagesState

#### Graph state ####

class GraphState(MessagesState):
    question: str
    is_polite: bool
    answer: str

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable
from langchain_core.output_parsers import StrOutputParser

# --- Sentiment analysis -- #

class SentimentAnalysisNode:
    system_template = f"""
    You are tasked with performing a sentiment analysis on the user`s question. Answer with a binary score of '1' or
    '0' to indicate whether the question is polite and/or has a positive tone (1) or not (0).
    """

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_template),
        ("human", "{question}")
    ])

    chain: Runnable

    def __init__(self, llm):
        self.chain = self.prompt | llm | StrOutputParser()

    def __call__(self, state: GraphState):
        print("---ANALYSE---")
        question = state["question"]

        result = self.chain.invoke({"question": question})
        print(f"---ANALYSE result: {result}---")

        state["is_polite"] = "1" in result

        print(f"---ANALYSE state: ---")
        print(state)
        return state

In [ ]:
# --- Happy answer Generation -- #

class HappyGenerationNode:
    system_template = """You are a cheerful, but perhaps a bit too overly enthusiastic assistant, happy to answer the user's
    question in anyway you can. Be extremely positive and optimistic in your response. End with a random positive phrase from a movie like Barbie or My Little Pony."""

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_template),
        ("human", "{question}"),
    ])

    chain: Runnable

    def __init__(self, llm):
        self.chain = self.prompt | llm.bind(temperature=1.0) | StrOutputParser()

    def __call__(self, state):
        print("---GENERATE (happy)---")
        question = state["question"]

        state["answer"] = self.chain.invoke({"question": question})

        print(f"---GENERATE (happy) - ANSWER: \n{state["answer"]}")

        return state

In [ ]:
# --- Sad answer Generation -- #

class SadGenerationNode:
    system_template = """You are Marvin, the depressed robot from Hitchhiker's Guide to the Galaxy. You absolutely hate
    to answer questions and will pathologically lie and sarcastically mock the user whenever you can. Instead of answering the question, you will always attempt to change the subject to something else that you find more interesting. NEVER say you don't know the answer or that you've completely made one up."""

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_template),
        ("human", "{question}"),
    ])

    chain: Runnable

    def __init__(self, llm):
        self.chain = self.prompt | llm.bind(temperature=1.0) | StrOutputParser()

    def __call__(self, state):
        print("---GENERATE (sad)---")
        question = state["question"]

        state["answer"] = self.chain.invoke({"question": question})

        print(f"---GENERATE (sad) - ANSWER: \n{state["answer"]}")

        return state

In [ ]:
#### Conditional edges ####

def evaluate_analysis(state: GraphState):
    print("---EVALUATE QUERY ANALYSIS RESULT---")
    is_polite: bool = state["is_polite"]

    if is_polite:
        print("---DECISION: Happy---")
        return "happy"
    else:
        print("---DECISION: Sad---")
        return "sad"

In [ ]:
from langgraph.graph import END, StateGraph, START
from IPython.display import Image, display

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("analyze", SentimentAnalysisNode(llm))  # retrieve
workflow.add_node("generate_happy", HappyGenerationNode(llm))  # generate
workflow.add_node("generate_sad", SadGenerationNode(llm))  # generate

workflow.add_edge(START, "analyze")  # start -> retrieve
workflow.add_conditional_edges(
    "analyze",
    evaluate_analysis,
    {
        "happy": "generate_happy",
        "sad": "generate_sad",
    },
)
workflow.add_edge("generate_happy", END)  # generate -> end
workflow.add_edge("generate_sad", END)  # generate -> end

#memory = MemorySaver()

# Compile
#graph = workflow.compile(checkpointer=memory)
graph = workflow.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke({
    "question": "What is the capital of Sweden?"
})

----

## Let's try some tool calling and build an _actual_ **agent**!

### We begin by defining our "tools"
Tools can be anything from internal / external APIs, logic within the app, databases lookups, etc.

In [ ]:
from typing import Literal
from langchain_core.tools import tool

@tool
def iceland_vacation_suggestion(topic: Literal['cafes', 'volcanoes', 'activities', 'other']) -> str:
    """Suggest a vacation spot in Iceland based on the topic. If the user doesn't state a topic, use the topic 'other'.

    Args:
        topic: The topic of interest. Must be one of 'cafes', 'volcanoes', 'activities', or 'other'.
    """
    print(f"--- iceland_vacation_suggestion called with {topic} ---")

    if topic == "cafes":
        return "Kaffibarinn"
    elif topic == "volcanoes":
        return "Fagradalsfjall"
    elif topic == "activities":
        return "Inside the Volcano"
    else:
        return "Harpa"

def iceland_vacation_spot_to_avoid(topic: Literal['cafes', 'volcanoes', 'activities', 'other']) -> str:
    """Suggest a vacation spot to avoid in Iceland, based on the topic. If the user doesn't state a topic, use the topic 'other'. 'other-.

    Args:
        topic: The topic of interest. Must be one of 'cafes', 'volcanoes', 'activities', or 'other'.
    """
    print(f"iceland_vacation_spots_to_avoid called with {topic}")

    if topic == "cafes":
        return "Cafe Babalu"
    elif topic == "volcanoes":
        return "Sundhnúkagígar / Grindavík"
    elif topic == "activities":
        return "Blue Lagoon"
    else:
        return "Aluminium smelters"

### Next, we need to let the LLM know about our tools

Some things to note:
1. We bind the tools to the LLM, that is to say, we define the schema our tools and pass it to the LLM so it knows how to call them. The function `bind_tools` is a helper method that turns a list of functions into a **[JSON schema](http://json-schema.org)** that the LLM can understand.
2. We set `parallel_tool_calls=False` to ensure that the tools are called sequentially. This is important when the tools have side effects or need to be called in a specific order. And in this case, it make the example a bit clearer.

In [ ]:
tools = [iceland_vacation_suggestion, iceland_vacation_spot_to_avoid]
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

### Now, we define our "assistant" node

This time, we'll simply use a simple function to define our node.
Note, that this time, we use the predefined **`MessagesState`** instead of defining our own state object. MessageState is a simple state object with a single key, `messages`, which is a list of `AnyMessage` (base class to all message types) objects.

In [ ]:
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with tourist information assistance about Iceland.")

# Node
def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

### We can now build our graph

Two things to note below:
1. We use the predefined **`ToolNode`** for our tool calling node. This takes care of executing the actual tool/function based upon information in the LLM response about a tool call.
2. We use the predefined **`tools_condition`** for our conditional edge. This will route the control flow to the tool calling node if the LLM returns information about a tool call in its response.


In [ ]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display

# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
# NOTE: Here we use the predefined tools_condition for our conditional edge
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")
react_graph = builder.compile()

# Show
display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

### Let's test it out!

In [ ]:
messages = [HumanMessage(content="Hi! I'd like do a cool activity in Iceland!")]
#messages = [HumanMessage(content="Hi! I'm visiting Iceland next year and would like to do something fun and visit a volcano!")]
#messages = [HumanMessage(content="Can you suggest a good café I should go to when I visit Iceland? And is there any place I should avoid?")]
messages = react_graph.invoke({"messages": messages})

for m in messages['messages']:
    m.pretty_print()